Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

/home/eren/tf_venv/tf_env_final/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [7]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [8]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.756275
Minibatch accuracy: 6.2%
Validation accuracy: 8.8%
Minibatch loss at step 50: 1.354232
Minibatch accuracy: 56.2%
Validation accuracy: 43.0%
Minibatch loss at step 100: 1.194917
Minibatch accuracy: 56.2%
Validation accuracy: 69.4%
Minibatch loss at step 150: 0.581749
Minibatch accuracy: 81.2%
Validation accuracy: 76.0%
Minibatch loss at step 200: 1.085258
Minibatch accuracy: 75.0%
Validation accuracy: 77.7%
Minibatch loss at step 250: 0.650908
Minibatch accuracy: 87.5%
Validation accuracy: 78.7%
Minibatch loss at step 300: 0.817924
Minibatch accuracy: 75.0%
Validation accuracy: 78.2%
Minibatch loss at step 350: 0.886611
Minibatch accuracy: 75.0%
Validation accuracy: 79.0%
Minibatch loss at step 400: 1.026942
Minibatch accuracy: 50.0%
Validation accuracy: 79.8%
Minibatch loss at step 450: 0.720814
Minibatch accuracy: 81.2%
Validation accuracy: 81.4%
Minibatch loss at step 500: 0.634805
Minibatch accuracy: 81.2%
Validation accuracy: 80.8%
Min

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [10]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    bias1 = tf.nn.relu(conv1 + layer1_biases)
    pool1 = tf.nn.max_pool(bias1, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    conv2 = tf.nn.conv2d(pool1, layer2_weights, [1, 1, 1, 1], padding='SAME')
    bias2 = tf.nn.relu(conv2 + layer2_biases)
    pool2 = tf.nn.max_pool(bias2, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    shape = pool2.get_shape().as_list()
    reshape = tf.reshape(pool2, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [11]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.669570
Minibatch accuracy: 0.0%
Validation accuracy: 10.7%
Minibatch loss at step 50: 1.139934
Minibatch accuracy: 56.2%
Validation accuracy: 51.4%
Minibatch loss at step 100: 0.934156
Minibatch accuracy: 75.0%
Validation accuracy: 72.5%
Minibatch loss at step 150: 0.611891
Minibatch accuracy: 81.2%
Validation accuracy: 77.1%
Minibatch loss at step 200: 0.833368
Minibatch accuracy: 81.2%
Validation accuracy: 77.1%
Minibatch loss at step 250: 0.694104
Minibatch accuracy: 81.2%
Validation accuracy: 78.0%
Minibatch loss at step 300: 0.764411
Minibatch accuracy: 68.8%
Validation accuracy: 80.5%
Minibatch loss at step 350: 0.906992
Minibatch accuracy: 81.2%
Validation accuracy: 80.0%
Minibatch loss at step 400: 1.110954
Minibatch accuracy: 62.5%
Validation accuracy: 80.1%
Minibatch loss at step 450: 0.754728
Minibatch accuracy: 81.2%
Validation accuracy: 81.0%
Minibatch loss at step 500: 0.388117
Minibatch accuracy: 87.5%
Validation accuracy: 81.4%
Mi

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

The CNN below is loosely inspired by the LeNet5 architecture.

In [12]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  size3 = ((image_size - patch_size + 1) // 2 - patch_size + 1) // 2
  layer3_weights = tf.Variable(tf.truncated_normal(
      [size3 * size3 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    # C1 input 28 x 28
    conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='VALID')
    bias1 = tf.nn.relu(conv1 + layer1_biases)
    # S2 input 24 x 24
    pool2 = tf.nn.avg_pool(bias1, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    # C3 input 12 x 12
    conv3 = tf.nn.conv2d(pool2, layer2_weights, [1, 1, 1, 1], padding='VALID')
    bias3 = tf.nn.relu(conv3 + layer2_biases)
    # S4 input 8 x 8
    pool4 = tf.nn.avg_pool(bias3, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    # F6 input 4 x 4
    shape = pool4.get_shape().as_list()
    reshape = tf.reshape(pool4, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [13]:
num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.830943
Minibatch accuracy: 6.2%
Validation accuracy: 8.8%
Minibatch loss at step 50: 1.540929
Minibatch accuracy: 37.5%
Validation accuracy: 46.2%
Minibatch loss at step 100: 1.201324
Minibatch accuracy: 56.2%
Validation accuracy: 60.1%
Minibatch loss at step 150: 0.819565
Minibatch accuracy: 81.2%
Validation accuracy: 66.4%
Minibatch loss at step 200: 1.137303
Minibatch accuracy: 75.0%
Validation accuracy: 71.7%
Minibatch loss at step 250: 0.970077
Minibatch accuracy: 62.5%
Validation accuracy: 73.3%
Minibatch loss at step 300: 0.920386
Minibatch accuracy: 68.8%
Validation accuracy: 76.8%
Minibatch loss at step 350: 1.064460
Minibatch accuracy: 68.8%
Validation accuracy: 75.6%
Minibatch loss at step 400: 1.175476
Minibatch accuracy: 62.5%
Validation accuracy: 75.7%
Minibatch loss at step 450: 0.939343
Minibatch accuracy: 75.0%
Validation accuracy: 78.2%
Minibatch loss at step 500: 0.553042
Minibatch accuracy: 87.5%
Validation accuracy: 77.4%
Min

Validation accuracy: 86.7%
Minibatch loss at step 4550: 0.677227
Minibatch accuracy: 81.2%
Validation accuracy: 86.9%
Minibatch loss at step 4600: 0.117078
Minibatch accuracy: 93.8%
Validation accuracy: 86.6%
Minibatch loss at step 4650: 0.578396
Minibatch accuracy: 87.5%
Validation accuracy: 86.9%
Minibatch loss at step 4700: 0.089257
Minibatch accuracy: 100.0%
Validation accuracy: 86.8%
Minibatch loss at step 4750: 0.634944
Minibatch accuracy: 87.5%
Validation accuracy: 86.6%
Minibatch loss at step 4800: 0.870204
Minibatch accuracy: 81.2%
Validation accuracy: 87.0%
Minibatch loss at step 4850: 0.840672
Minibatch accuracy: 81.2%
Validation accuracy: 86.6%
Minibatch loss at step 4900: 0.770759
Minibatch accuracy: 75.0%
Validation accuracy: 86.8%
Minibatch loss at step 4950: 0.358050
Minibatch accuracy: 87.5%
Validation accuracy: 87.1%
Minibatch loss at step 5000: 0.212722
Minibatch accuracy: 87.5%
Validation accuracy: 86.9%
Minibatch loss at step 5050: 0.720742
Minibatch accuracy: 81.2

Validation accuracy: 88.3%
Minibatch loss at step 9050: 0.482547
Minibatch accuracy: 87.5%
Validation accuracy: 87.2%
Minibatch loss at step 9100: 0.882117
Minibatch accuracy: 81.2%
Validation accuracy: 88.5%
Minibatch loss at step 9150: 0.085915
Minibatch accuracy: 100.0%
Validation accuracy: 88.4%
Minibatch loss at step 9200: 0.261157
Minibatch accuracy: 93.8%
Validation accuracy: 88.0%
Minibatch loss at step 9250: 0.402739
Minibatch accuracy: 87.5%
Validation accuracy: 88.1%
Minibatch loss at step 9300: 0.472858
Minibatch accuracy: 81.2%
Validation accuracy: 87.7%
Minibatch loss at step 9350: 0.510440
Minibatch accuracy: 81.2%
Validation accuracy: 88.2%
Minibatch loss at step 9400: 0.123796
Minibatch accuracy: 87.5%
Validation accuracy: 88.2%
Minibatch loss at step 9450: 0.606175
Minibatch accuracy: 81.2%
Validation accuracy: 87.8%
Minibatch loss at step 9500: 0.017569
Minibatch accuracy: 100.0%
Validation accuracy: 88.0%
Minibatch loss at step 9550: 0.417577
Minibatch accuracy: 81.

Minibatch loss at step 13500: 0.980159
Minibatch accuracy: 75.0%
Validation accuracy: 88.7%
Minibatch loss at step 13550: 0.568776
Minibatch accuracy: 87.5%
Validation accuracy: 89.0%
Minibatch loss at step 13600: 0.329116
Minibatch accuracy: 87.5%
Validation accuracy: 89.0%
Minibatch loss at step 13650: 0.117862
Minibatch accuracy: 100.0%
Validation accuracy: 88.8%
Minibatch loss at step 13700: 0.353817
Minibatch accuracy: 87.5%
Validation accuracy: 89.3%
Minibatch loss at step 13750: 0.639251
Minibatch accuracy: 81.2%
Validation accuracy: 89.0%
Minibatch loss at step 13800: 0.359632
Minibatch accuracy: 87.5%
Validation accuracy: 89.1%
Minibatch loss at step 13850: 0.180378
Minibatch accuracy: 93.8%
Validation accuracy: 89.4%
Minibatch loss at step 13900: 0.285172
Minibatch accuracy: 93.8%
Validation accuracy: 89.2%
Minibatch loss at step 13950: 0.177720
Minibatch accuracy: 93.8%
Validation accuracy: 89.1%
Minibatch loss at step 14000: 0.203581
Minibatch accuracy: 93.8%
Validation acc

Minibatch loss at step 17950: 0.408146
Minibatch accuracy: 87.5%
Validation accuracy: 89.0%
Minibatch loss at step 18000: 0.086095
Minibatch accuracy: 100.0%
Validation accuracy: 89.6%
Minibatch loss at step 18050: 0.383299
Minibatch accuracy: 93.8%
Validation accuracy: 88.8%
Minibatch loss at step 18100: 0.550035
Minibatch accuracy: 81.2%
Validation accuracy: 89.5%
Minibatch loss at step 18150: 0.481997
Minibatch accuracy: 87.5%
Validation accuracy: 89.2%
Minibatch loss at step 18200: 1.025181
Minibatch accuracy: 81.2%
Validation accuracy: 89.3%
Minibatch loss at step 18250: 0.318066
Minibatch accuracy: 93.8%
Validation accuracy: 89.4%
Minibatch loss at step 18300: 0.738392
Minibatch accuracy: 75.0%
Validation accuracy: 88.9%
Minibatch loss at step 18350: 0.631945
Minibatch accuracy: 75.0%
Validation accuracy: 89.3%
Minibatch loss at step 18400: 0.146302
Minibatch accuracy: 93.8%
Validation accuracy: 89.3%
Minibatch loss at step 18450: 0.092371
Minibatch accuracy: 100.0%
Validation ac

The accuracy is good, but not as good as the 3-layer network from the previous assignment.

The next version of the net uses dropout and learning rate decay:

In [14]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64
beta_regul = 1e-3
drop_out = 0.5

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  global_step = tf.Variable(0)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  size3 = ((image_size - patch_size + 1) // 2 - patch_size + 1) // 2
  layer3_weights = tf.Variable(tf.truncated_normal(
      [size3 * size3 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_hidden], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer5_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data, keep_prob):
    # C1 input 28 x 28
    conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='VALID')
    bias1 = tf.nn.relu(conv1 + layer1_biases)
    # S2 input 24 x 24
    pool2 = tf.nn.avg_pool(bias1, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    # C3 input 12 x 12
    conv3 = tf.nn.conv2d(pool2, layer2_weights, [1, 1, 1, 1], padding='VALID')
    bias3 = tf.nn.relu(conv3 + layer2_biases)
    # S4 input 8 x 8
    pool4 = tf.nn.avg_pool(bias3, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    # F5 input 4 x 4
    shape = pool4.get_shape().as_list()
    reshape = tf.reshape(pool4, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden5 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    # F6
    drop5 = tf.nn.dropout(hidden5, keep_prob)
    hidden6 = tf.nn.relu(tf.matmul(hidden5, layer4_weights) + layer4_biases)
    drop6 = tf.nn.dropout(hidden6, keep_prob)
    return tf.matmul(drop6, layer5_weights) + layer5_biases
  
  # Training computation.
  logits = model(tf_train_dataset, drop_out)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))

  # Optimizer.
  learning_rate = tf.train.exponential_decay(0.05, global_step, 1000, 0.85, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset, 1.0))
  test_prediction = tf.nn.softmax(model(tf_test_dataset, 1.0))

In [15]:
num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.497251
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.159835
Minibatch accuracy: 25.0%
Validation accuracy: 17.6%
Minibatch loss at step 100: 1.909182
Minibatch accuracy: 50.0%
Validation accuracy: 49.8%
Minibatch loss at step 150: 1.290364
Minibatch accuracy: 56.2%
Validation accuracy: 45.1%
Minibatch loss at step 200: 2.355863
Minibatch accuracy: 37.5%
Validation accuracy: 60.8%
Minibatch loss at step 250: 1.587837
Minibatch accuracy: 37.5%
Validation accuracy: 59.1%
Minibatch loss at step 300: 1.335035
Minibatch accuracy: 43.8%
Validation accuracy: 68.5%
Minibatch loss at step 350: 1.398773
Minibatch accuracy: 56.2%
Validation accuracy: 69.9%
Minibatch loss at step 400: 1.116001
Minibatch accuracy: 62.5%
Validation accuracy: 69.1%
Minibatch loss at step 450: 1.138585
Minibatch accuracy: 68.8%
Validation accuracy: 74.1%
Minibatch loss at step 500: 0.820536
Minibatch accuracy: 75.0%
Validation accuracy: 72.8%
Mi

Validation accuracy: 86.1%
Minibatch loss at step 4550: 0.840164
Minibatch accuracy: 75.0%
Validation accuracy: 86.1%
Minibatch loss at step 4600: 0.205814
Minibatch accuracy: 93.8%
Validation accuracy: 86.0%
Minibatch loss at step 4650: 0.620081
Minibatch accuracy: 87.5%
Validation accuracy: 86.0%
Minibatch loss at step 4700: 0.163851
Minibatch accuracy: 93.8%
Validation accuracy: 86.2%
Minibatch loss at step 4750: 0.407528
Minibatch accuracy: 87.5%
Validation accuracy: 86.2%
Minibatch loss at step 4800: 0.611325
Minibatch accuracy: 81.2%
Validation accuracy: 86.0%
Minibatch loss at step 4850: 0.726373
Minibatch accuracy: 81.2%
Validation accuracy: 86.0%
Minibatch loss at step 4900: 0.884739
Minibatch accuracy: 68.8%
Validation accuracy: 86.3%
Minibatch loss at step 4950: 0.388297
Minibatch accuracy: 81.2%
Validation accuracy: 86.3%
Minibatch loss at step 5000: 0.287714
Minibatch accuracy: 93.8%
Validation accuracy: 86.1%
Test accuracy: 91.8%


Well, the accuracy is worst. This net has many meta parameters and I don't feel comfortable in tuning them randomly. I should probably change the depth and make it different between the layers, since it looks like the increasing number of feature maps is a key design item.

I will do so in a next version.